Bare minimum for 
- consumption of model.joblib
- export of properly formatted .json

# imports

## dependencies

In [1]:
import joblib
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [2]:
#!pip install scikit-learn==0.24.2

## files

In [3]:
model_path = "../data/models_anime_map_knn_model.joblib"

In [4]:
def get_anime():
    anime_df_relevant_PG = pd.read_csv("../data/anime_df_relevant_PG.csv")
    return anime_df_relevant_PG.rename(columns={'MAL_ID' : 'anime_id'})

In [5]:
def get_data(name_file):
    data = pd.read_csv(f'../data/{name_file}.csv')
    return data


In [6]:
def get_model(path):
    return joblib.load(model_path)

In [7]:
def process_data(name_file):
    data_users_df = get_data(name_file)
    data_users_df['rating'] = data_users_df['rating']/10
    
    anime_df_relevant_PG = get_anime()
    anime_name_df = anime_df_relevant_PG[['anime_id','Name']]
    data_users_df_merge = data_users_df.merge(anime_name_df, on = 'anime_id', how='inner')
    pivot_df = data_users_df_merge.pivot_table(index='anime_id',columns='user_id',values='rating').fillna(0)
    
    anime_Genres_df = anime_df_relevant_PG[['anime_id','Genres']]
    anime_Genres_df_encoded = pd.concat(objs = [anime_Genres_df.drop(columns = 'Genres', axis =1), anime_Genres_df['Genres'].str.get_dummies(sep=", ")], axis = 1)
    anime_Genres_df_encoded = anime_Genres_df_encoded.set_index('anime_id')
    
    pivot_df = pivot_df.merge(anime_Genres_df_encoded, how='inner',left_index=True, right_index=True)
    anime_name_pivot_df = data_users_df_merge[['anime_id','Name']].drop_duplicates()
    anime_name_pivot_df = anime_name_pivot_df.sort_values('anime_id')
    anime_name_pivot_df = anime_name_pivot_df.reset_index().drop(columns = 'index')
    
    return pivot_df, anime_name_pivot_df

# predict

In [8]:
def recommendation_10PlusRatings(anime_name, nb_recomendation = 10):
    pivot_df, anime_name_pivot_df = process_data('active_users_df_10PlusRatings_partial')
    model = get_model(model_path)
    index_nb = anime_name_pivot_df.index[anime_name_pivot_df['Name'] == anime_name].tolist()[0]
    distances, indices = model.kneighbors(pivot_df.iloc[index_nb,:].values.reshape(1, -1), n_neighbors = nb_recomendation + 1)

    prediction = []
    for i in range(1, len(distances.flatten())):
        prediction.append([pivot_df.index[indices.flatten()[i]],distances.flatten()[i]])
    results = {}
    for i in range(len(prediction)):
        anime_name = anime_name_pivot_df.query(f'anime_id == {prediction[i][0]}').iloc[0].Name
        distance = prediction[i][1]
        results[f'{anime_name}'] = distance
    return results

In [13]:
predict_Naruto = recommendation_10PlusRatings('Naruto', 30)

In [14]:
predict_Naruto

{'Naruto: Shippuuden': 0.18049078542744768,
 'Death Note': 0.32724375187989996,
 'Bleach': 0.35922338431707146,
 'Shingeki no Kyojin': 0.3764965221555553,
 'Fullmetal Alchemist: Brotherhood': 0.4092187913883153,
 'Code Geass: Hangyaku no Lelouch': 0.4141593159792041,
 'Sword Art Online': 0.4145034622526954,
 'One Piece': 0.4324693299705893,
 'Fairy Tail': 0.43374727465720364,
 'One Punch Man': 0.440672137299862,
 'Code Geass: Hangyaku no Lelouch R2': 0.44317776537599496,
 'Tokyo Ghoul': 0.4478750610328671,
 'Boku no Hero Academia': 0.4509773475530082,
 'Fullmetal Alchemist': 0.45734048611945144,
 'Ao no Exorcist': 0.459692800542125,
 'Hunter x Hunter (2011)': 0.4624201558967598,
 'Angel Beats!': 0.4655259992943921,
 'Dragon Ball Z': 0.4667113626750077,
 'Boku no Hero Academia 2nd Season': 0.4783955337192296,
 'No Game No Life': 0.4788718820530793,
 'Toradora!': 0.4801106307799001,
 'Highschool of the Dead': 0.48100086207446635,
 'Nanatsu no Taizai': 0.48689829546741004,
 'Shingeki no K

# Export model.joblib as pickle / sav file

In [97]:
model_joblib = get_model(model_path)

In [98]:
model_joblib

NearestNeighbors(metric='cosine')

In [99]:
import pickle

In [102]:
pickle.dump(model_joblib, open('model.sav',"wb"))

In [103]:
ls

FrontEnd_Explo.ipynb  model.sav


In [104]:
pwd


'/home/mijka/code/mijkami/AnimeMap_front/notebook'